## Data Genrator 

In [1]:
from keras.utils import to_categorical
from PIL import Image
import numpy as np
import pandas as pd
from keras.layers import Input,Activation,Add
from keras.layers import Conv2D,Dense,MaxPooling2D
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.layers import Flatten,BatchNormalization
from keras.layers import Dropout
import keras
from keras.optimizers import Adam

In [2]:
df = pd.read_csv("/home/andy/Desktop/sken_project/facial_emotion_detection/datasets/clean_imfdb.csv",index_col=None)

In [3]:
df.head()

,file_name,sex,emotion,age,image_path
0,Leelavathi_1.jpg,female,anger,old,/home/andy/Desktop/sken_project/facial_emotion...
1,Leelavathi_10.jpg,female,happiness,old,/home/andy/Desktop/sken_project/facial_emotion...
2,Leelavathi_11.jpg,female,anger,old,/home/andy/Desktop/sken_project/facial_emotion...
3,Leelavathi_12.jpg,female,anger,old,/home/andy/Desktop/sken_project/facial_emotion...
4,Leelavathi_15.jpg,female,surprise,old,/home/andy/Desktop/sken_project/facial_emotion...


In [4]:
df.columns

Index(['file_name', 'sex', 'emotion', 'age', 'image_path'], dtype='object')

In [5]:
TRAIN_TEST_SPLIT = 0.8
IM_WIDTH = 480
IM_HEIGHT = 640

dataset_dict = {
    "age_id":{
        0: "old",
        1: "middle",
        2: "young",
        3: "child"
    },
    "gender_id":{
        0: "male",
        1: "female"
    },
    "emotion_id":{
        0: "neutral",
        1: "happiness",
        2: "surprise",
        3: "anger",
        4: "sadness",
        5: "disgust",
        6: "fear"
    }
}

In [6]:
dataset_dict['gender_alias'] = dict((g,i) for i,g in dataset_dict['gender_id'].items())
dataset_dict['age_alias'] = dict((g,i) for i,g in dataset_dict["age_id"].items())
dataset_dict['emotion_alias'] = dict((g,i) for i,g in dataset_dict['emotion_id'].items())

In [7]:
dataset_dict

{'age_id': {0: 'old', 1: 'middle', 2: 'young', 3: 'child'},
 'gender_id': {0: 'male', 1: 'female'},
 'emotion_id': {0: 'neutral',
  1: 'happiness',
  2: 'surprise',
  3: 'anger',
  4: 'sadness',
  5: 'disgust',
  6: 'fear'},
 'gender_alias': {'male': 0, 'female': 1},
 'age_alias': {'old': 0, 'middle': 1, 'young': 2, 'child': 3},
 'emotion_alias': {'neutral': 0,
  'happiness': 1,
  'surprise': 2,
  'anger': 3,
  'sadness': 4,
  'disgust': 5,
  'fear': 6}}

In [8]:
class ImfdbDataGenrator():
    """
    Data generator for IMFDB dataset. This class should be used when training our custom multi output model
    """
    def __init__(self,df):
        self.df = df
    
    def generate_split_indexes(self):
        p = np.random.permutation(len(self.df))
        train_up_to = int(len(self.df) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]
        
        train_up_to = int(train_up_to* TRAIN_TEST_SPLIT)
        train_idx,valid_idx = train_idx[:train_up_to],train_idx[train_up_to:]
        
        #convert alias to id
        self.df['gender_id'] = self.df['sex'].map(lambda gen: dataset_dict['gender_alias'][gen])
        self.df['age_id'] = self.df['age'].map(lambda age: dataset_dict['age_alias'][age])
        self.df['emotion_id'] = self.df['emotion'].map(lambda emo: dataset_dict['emotion_alias'][emo])
        return train_idx,valid_idx,test_idx
    
    def preprocess_image(self,image_path):
        """
        Used for image preprocessing 
        """
        img = Image.open(image_path)
        img = img.resize((IM_WIDTH,IM_HEIGHT),Image.ANTIALIAS)
        img = np.array(img)/255.0
        return img
    def grenerate_images(self,image_idx,is_training,batch_size=16):
        """
        Used to genrate batch with images 
        """
        images,ages,emotions,genders = [],[],[],[]
        while True:
            for idx in image_idx:
                person = self.df.iloc[idx]
                age = person['age_id']
                emotion = person['emotion_id']
                gender = person['gender_id']
                file = person['image_path']
                im = self.preprocess_image(file)
                ages.append(to_categorical(age,len(dataset_dict['age_id'])))
                genders.append(to_categorical(gender,len(dataset_dict['gender_id'])))
                emotions.append(to_categorical(emotion,len(dataset_dict['emotion_id'])))
                images.append(im)
                
                if len(images) >= batch_size:
                    yield np.array(images),[np.array(genders),np.array(ages),np.array(emotions)]
                    images,ages,emotions,genders = [],[],[],[]
            if not is_training:
                break

In [9]:
data_gen = ImfdbDataGenrator(df)

In [10]:
data_gen.df.columns

Index(['file_name', 'sex', 'emotion', 'age', 'image_path'], dtype='object')

In [11]:
train_idx,valid_idx,test_idx = data_gen.generate_split_indexes()

In [12]:
data_gen.df.columns

Index(['file_name', 'sex', 'emotion', 'age', 'image_path', 'gender_id',
       'age_id', 'emotion_id'],
      dtype='object')

In [13]:
class ImfdbOutputModel():
    """
    This class is used to genrate our multi-output model which is a vgg16 with its 4 blocks and then we 
    add our three output layers with the vgg layers freesed
    """
    def __init__(self,custom_model_path):
        self.model = keras.models.load_model(custom_model_path)
    def get_model(self):
        for layer in self.model.layers[:15]:
            layer.trainable = False
        return self.model

In [14]:
custom_model_obj = ImfdbOutputModel("/home/andy/Desktop/sken_project/facial_emotion_detection/model/custom_model")

In [15]:
model = custom_model_obj.get_model()

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 640, 480, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 640, 480, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 640, 480, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 320, 240, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

In [17]:
init_lr = 1e-4
epochs =50
opt = Adam(lr=init_lr,decay=init_lr/epochs)

In [18]:
model.compile(optimizer=opt,
             loss={
                 "sex_out": 'categorical_crossentropy',
                 "age_out": 'categorical_crossentropy',
                 "emotion": 'categorical_crossentropy'
             },
             loss_weights={
                 "sex_out": 0.1,
                 "age_out": 1.5,
                 "emotion": 1.5
             },
             metrics={
                 "sex_out": 'categorical_accuracy',
                 "age_out": 'categorical_accuracy',
                 "emotion": "categorical_accuracy"
             })

In [19]:
from keras.callbacks import ModelCheckpoint
batch_size = 32
valid_batch_size = 32

In [20]:
train_gen = data_gen.grenerate_images(train_idx,is_training=True,batch_size=batch_size)
valid_gen = data_gen.grenerate_images(valid_idx,is_training=True,batch_size=batch_size)

In [21]:
callbacks = [ModelCheckpoint("./model_checkpoint",monitor='val_loss')]

In [22]:
history = model.fit_generator(generator=train_gen,
                             steps_per_epoch=len(train_idx)//batch_size,
                              verbose=True,
                             epochs=epochs,
                             callbacks=callbacks,
                             validation_data=valid_gen,
                             validation_steps=len(valid_idx)//valid_batch_size)

/home/andy/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
 84/609 [===>..........................] - ETA: 1:47:39 - loss: 5.5860 - sex_out_loss: 0.7310 - age_out_loss: 1.4505 - emotion_loss: 2.2247 - sex_out_categorical_accuracy: 0.6178 - age_out_categorical_accuracy: 0.4262 - emotion_categorical_accuracy: 0.2128

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/andy/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-86d80d74bf11>", line 7, in <module>
    validation_steps=len(valid_idx)//valid_batch_size)
  File "/home/andy/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1861, in fit_generator
    initial_epoch=initial_epoch)
  File "/home/andy/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "/home/andy/.local/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/andy/.local/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 855, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "/ho

KeyboardInterrupt: 